# Intelligence artificielle - Automne 2025 - Laboratoire 07

## _Problèmes de satisfaction des contraintes (CSP)_

## Introduction

**Les problèmes de satisfaction des contraintes** sont définis par :
* un ensemble de **variables** discrètes ;
* un **domaine (plage) de valeurs** pour chaque variable ;
* un ensemble de **contraintes** sur les variables.

## Des définitions au savoir-faire

In [ ]:
# Useful libraries:
from copy import copy, deepcopy
from itertools import combinations

Du point de vue pythonien :
  - chaque variable sera représentée par une chaîne de caractères,
  
```python
    vars_list = ['A', 'B', 'C']
```

   - l'ensemble des domaines sera un dictionnaire avec une entrée pour chaque variable:
     + la clé sera le nom de la variable,
     + la valeur est donnée par les valeurs de la plage de cette variable

```python
    domains_dict = {'A': [1, 2, 3], 'B': [1, 5, 9], 'C': [-2, -1]}
```

   - une contrainte sera représentée par un `tuple` composé d':
     + un `list` des variables impliquées dans la contrainte
     + une fonction anonyme qui retourne `True` ou`False`

```python
    constraints = [(['A', 'B', 'C'], lambda a, b, c: a + b + c == 0)]
```

La **solution** sera représentée par un dictionnaire qui indique une valeur pour chaque variable (par exemple `{'A': 1,'B': 1,'C': -2}`).

Le **coût** est donné par le nombre de contraintes ignorées par la solution.

Problème mathématique

In [ ]:
VarsA = ["A", "B", "C", "D", "E"]
DomainsA = {v: [i for i in range(10)] for v in VarsA}
ConstraintsA = [(list(p), lambda x,y: x != y) for p in combinations(VarsA, 2)] # all the values are different
ConstraintsA.append((["A","B"], lambda a, b: a + b == 10))
ConstraintsA.append((["B","D"], lambda b, d: b + d == 6))
ConstraintsA.append((["C"], lambda c: c < 5))
ConstraintsA.append((["A"], lambda a: a > 5))
ConstraintsA.append((["A","B","C","D","E"], lambda a, b, c, d, e: a + b + c + d + e == 30))
MathProblem = {"Vars": VarsA, "Domains": DomainsA, "Constraints": ConstraintsA}

Problème de coloration des cartes

In [ ]:
VarsC = ["France", "Germany", "Loux", "Belgium", "Netherlands"]
DomainsC = {v: ["blue", "red", "yellow", "green"] for v in VarsC}
ConstraintsC = []
for (a, b) in [("France", "Germany"), ("France", "Belgium"), ("France", "Loux"),
               ("Belgium", "Netherlands"), ("Belgium", "Loux"), ("Belgium", "Germany"),
               ("Loux", "Germany"), ("Netherlands", "Germany")]:
    ConstraintsC.append(([a, b], lambda a, b: a != b))
ColoringProblem = {"Vars": VarsC, "Domains": DomainsC, "Constraints": ConstraintsC}

### Tâche 0

Implémentez la fonction `get_constraints` qui reçoit une variable `var` et une liste de contraintes `constraints` et retournez seulement les contraintes qui impliquent la variable donnée.

_Résultat attendu:_

```
Constraints = [(["A", "B"], lambda a,b: a<b), (["A"], lambda a: a<5)]
get_constraints("B", Constraints)
==> [(["A", "B"], lambda a,b: a<b)]
get_constraints("A", Constraints)
==> [(["A", "B"], lambda a,b: a<b), (["A"], lambda a: a<5)]
```

In [ ]:
def get_constraints(var, constraints):
    # TODO
    return []

get_constraints("France", ConstraintsC) # => [(['France', 'Germany'], ...), (['France', 'Belgium'], ...), (['France', 'Loux'], ...)]

### Tâche 1

Implémentez la fonction `fixed_constraints` qui reçoit une `solution` comme solution partielle et un ensemble de contraintes `constraints`. La fonction retourne seulement les contraintes qui peuvent être évaluées (c'est-à-dire que les variables impliquées se trouvent dans la solution partielle).

In [ ]:
def fixed_constraints(solution, constraints):
    # TODO
    return []

print(fixed_constraints({"France": "blue", "Belgium": "green"}, ConstraintsC)) # => [(['France', 'Belgium'], ...)]
print(fixed_constraints({"A": "1", "C": "2"}, ConstraintsA)) # => [(['A', 'C'], ...), (['C'], ...), (['A'], ...)]

### Tâche 2

Implémentez la fonction `check_constraint` qui reçoit comme contrainte la variable `constraint`, une solution partielle `solution` et retourne `True` si la solution respecte la contrainte et `False` sinon.

_**Conseil:**_ pour appeler une fonction connaissant la liste de ses paramètres utilisez `f(*args)`, où `args` est la liste des arguments.

In [ ]:
def check_constraint(solution, constraint):
    # TODO
    return False

print(check_constraint({"France": "blue", "Belgium": "green"}, ConstraintsC[1])) # => True
print(check_constraint({"France": "blue", "Belgium": "blue"}, ConstraintsC[1])) # => False
print(check_constraint({"C": 10, "A": 10}, ConstraintsA[-2])) # => True
print(check_constraint({"C": 10, "A": 3}, ConstraintsA[-2])) # => False

### Task 3: Algorithme PCSP

Complétez ce qui manque dans l'algorithme du PCSP.

* `vars` - les variables qui restent à vérifier
* `domains` - les domaines pour les variables qui restent à vérifier, avec les valeurs qui restent à vérifier pour chaque variable
* `constrains` - la liste des contraintes
* `acceptable_cost` - le coût pour lequel la solution est jugée satisfaisante
* `solution` - la solution partielle, contenant des valeurs pour les variables vérifiées jusqu'à présent
* `cost` - le coût des contraintes non satisfaites

La valeur de retour de la fonction est `True` si une solution complète satisfaisante a été trouvée (voir coût acceptable), et `False` dans le cas contraire.

Deux variables globales sont utilisées:

* `best_cost` - le meilleur (le plus petit) coût jusqu'à présent dans l'exploration, pour une solution complète
* `best_solution` - la solution correspondant au meilleur coût

In [ ]:
def PCSP(vars, domains, constraints, acceptable_cost, solution, cost):
    global best_solution
    global best_cost
    if not vars:
        # If there are no more variables, we have reached a better solution
        print("New best: " + str(cost) + " - " + str(solution))
        # TODO: save the newly discovered solution
        # TODO: if it is good enough, the function returns True
    
    elif not domains[vars[0]]:
        # If there are no more values in the domains, the searching process ends
        return False
    elif cost == best_cost:
        # If we have already reached a cost identical with the one of the best solution, we will not go any further
        return False
    else:
        # TODO: Choose the first variable and the first value in the domains
        var = vars[0]
        val = domains[var].pop(0)

        # TODO: Build the new solution
        new_solution = {}

        # TODO: We get the list of constraints that can be evaluated
        # TODO: We compute the cost of the new partial solution (each constraint ignored = 1)
        new_cost = 0

        # We check if the new cost is lower than the best cost
        if new_cost < best_cost:
            # TODO:
            pass
            # If the new cost is lower than the best known, we solve for the rest of the variables
            # If the recursive call returns True, a good enough solution has been found, so we return True
        
        # Check for the rest of the values
        # TODO:
    
# A wrapper that instantiates global variables
def run_pcsp(problem, acceptable_cost):
    global best_solution
    global best_cost

    [vars, domains, constraints] = [problem[e] for e in ["Vars", "Domains", "Constraints"]]
    
    best_solution = {}
    best_cost = len(constraints)

    if PCSP(vars, deepcopy(domains), constraints, acceptable_cost, {}, 0):
        print("Best found: " + str(best_cost) + " - " + str(best_solution))
    else:
        print("Acceptable solution not found; " + "Best found: " + str(best_cost) + " - " + str(best_solution))
        
# Run the algorithm
run_pcsp(MathProblem, 1)
run_pcsp(ColoringProblem, 1)

Résultats prévus pour les chiffres:

```
New best: 14 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}
New best: 10 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 1}
New best: 8 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}
New best: 7 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 2}
New best: 6 - {'A': 0, 'B': 0, 'C': 0, 'D': 6, 'E': 1}
New best: 5 - {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 3}
New best: 4 - {'A': 0, 'B': 0, 'C': 1, 'D': 6, 'E': 2}
New best: 3 - {'A': 0, 'B': 1, 'C': 2, 'D': 5, 'E': 3}
New best: 2 - {'A': 2, 'B': 8, 'C': 4, 'D': 7, 'E': 9}
New best: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
Best found: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
```

Résultats attendus pour les couleurs des pays:

```
New best:  8  -  {'Loux': 'blue', 'Belgium': 'blue', 'Netherlands': 'blue', 'Germany': 'blue', 'France': 'blue'}
New best:  6  -  {'Loux': 'blue', 'Belgium': 'blue', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
New best:  4  -  {'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'blue', 'Germany': 'blue', 'France': 'blue'}
New best:  3  -  {'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'yellow', 'Germany': 'blue', 'France': 'blue'}
New best:  2  -  {'Loux': 'red', 'Belgium': 'red', 'Netherlands': 'yellow', 'Germany': 'blue', 'France': 'blue'}
New best:  1  -  {'Loux': 'red', 'Belgium': 'yellow', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
Best found: 1  -  {'Loux': 'red', 'Belgium': 'yellow', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
```